- competition/dataset : [https://www.kaggle.com/c/statoil-iceberg-classifier-challenge/overview](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge/overview)
- date : 2021/02/09
- original : [https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d](https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d)

## Keras Model for Beginners (0.210 on LB)+EDA+R&D

**✏ 필사 1회** 

### 1. Description
표류하는 빙산은 캐나다의 동부 해안과 같은 지역에서의 항해 및 활동에 위협을 줍니다.  

현재, 여러 기관과 회사들은 환경 조건 모니터링하고 빙산에 의한 위험을 평가하기 위해 항공 정찰과 해안 기반 지원 사업을 하고 있습니다. 그러나 날씨가 특히 험한 외딴 지역에서는 이런 방법들이 불가능하며, 실현 가능한 모니터링 방법은 위성을 통하는 것 뿐입니다.  

전세계에서 활동하고 있는 국제 에너지 회사 Statoil은 C-CORE 등의 기업들과 긴밀하게 협력해왔습니다. C-CORE는 30년이 넘는 위성 데이터를 가지고 있으며, 컴퓨터 비전 기반의 감시 시스템을 구축했습니다. 안전하고 효과적인 운영을 위해 Statoil은 가능한 한 빨리 위협적인 빙산을 더 정확하게 감지하고 구별하기 위한 머신러닝 사용법에 대한 새로운 관점을 얻고자 합니다.  

이 대회에서는 멀리서 감지된 타겟이 배인지 빙산인지를 자동으로 구별하는 알고리즘을 설계하는 것이 목표입니다. 개선은 안전 작업 환경 유지를 위한 비용을 절감하는데 도움이 될 것입니다.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from os.path import join as opj

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
# Load the data
train = pd.read_json('data/train.json')
test = pd.read_json('data/test.json')

### 2. Intro about the Data
Sentinet -1 위성은 지구로부터 약 680km 떨어져 있습니다. 특정 입사각에서 신호의 펄스를 보내고 다시 recoding합니다. 기본적으로 이러한 반사 신호를 backscatter(후방산란)라고 합니다. 우리에게 제공된 데이터는 다음 식에 의해 주어진 후방산란계수의 전형적인 형태입니다:  
$$\sigma_{o}(dB) = \beta_{o}(dB) + 10~log10[sin(ip)/sin(ic)]$$  

ip: 특정 픽셀의 입사각  
ic: 이미지 중심부의 입사각  
K: 상수  

우리는 데이터에서 $\sigma_{o}$를 바로 확인할 수 있습니다.

##### Now coming to the features of $\sigma_{o}$
일반적으로 $\sigma_{o}$는 신호가 산란되는 표면에 따라 달라집니다. 예를 들어, 특정 입사각에서는 다음과 같이 변합니다:  

ㅤ |WATER | SETTLEMENTS | AGRICULTURE | BARREN
-- | -- | -- | -- | --
**HH** | -27.001 | 2.70252 | -12.7952 | -17.25790909
**HV** | -28.035 | -20.2665 | -21.4471 | -20.019  

HH의 경우 값이 크게 변화하지만 HV는 그렇지 않습니다. **배에 대한 산란에 대한 데이터는 없지만 금속 물질이기 때문에 얼음과는 매우 달라야 합니다.**

##### WTF is HH HV?
Sentinel 위성은 RISTSAT(인도의 원격 감지 위성)와 같습니다. H polarization만 전송하고 V polarization은 전송하지 않습니다. H-ping이 산란되면 물질의 polarization이 바뀌고 H와 V가 혼합되어 돌아옵니다. **Sentinel 위성은 H-transmetter만 가지고 있기 때문에 돌아오는 신호는 HH와 HV 형태 뿐입니다.** Sentinel은 V-ping transmitter는 가지고 있지 않기 때문에 VV가 없는 이유에 대해서는 묻지 마십시오.  

먼저, 두 band를 모두 추출하고 평균값을 세 번째 채널로 사용하여 3-channel RGB equvalent를 생성하겠습니다.

In [3]:
train.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [4]:
# 트레이닝 데이터 생성
# HH, HV, 평균에 대한 세 개의 band 생성
x_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
x_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])
x_train = np.concatenate(
    [x_band_1[:, :, :, np.newaxis],
     x_band_2[:, :, :, np.newaxis],
     ((x_band_1 + x_band_2)/2)[:, :, :, np.newaxis]], axis=-1
)

💡 np.newaxis: 새로운 차원 추가

In [5]:
# 빙산 데이터 시각화
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):
    data = [go.Surface(z=c)]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=600,
        height=500,
        margin=dict(l=65, r=50, b=65, t=90)
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

plotmy3d(x_band_1[12, :, :], 'iceberg')

레이더 데이터에서 빙산의 형태가 위와 같이 산처럼 될 것이라는 점을 기억해야 합니다. 실제 이미지가 아니라 레이더에 의한 산란이기 때문에 peak와 왜곡이 있을 수 있습니다. 배의 형태는 점과 같으며 가늘고 뾰족한 점이 될 것입니다. 여기서 구조적 차이가 발생하고, 우리는 CNN을 사용하여 차이점을 이용할 수 있습니다. 레이더의 backscatter를 이용하여 합성물을 만들 수 있다면 큰 도움이 될 것입니다.

In [6]:
train.loc[train['is_iceberg'] == 0, ]

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0
7,56929c16,"[-24.989119, -27.755224, -25.817074, -24.98927...","[-27.755173, -26.732174, -28.124943, -31.83772...",41.1769,0
...,...,...,...,...,...
1599,04e11240,"[-30.999878, -29.976866, -28.233906, -29.50732...","[-27.847719, -28.233864, -24.712077999999998, ...",na,0
1600,c7d6f6f8,"[-25.31155, -26.511555, -28.694487, -27.180115...","[-29.563713, -28.290375, -26.839405, -28.29046...",na,0
1601,bba1a0f1,"[-18.141895, -18.141844, -19.01737, -19.701599...","[-25.305355, -29.387701, -28.963863, -26.16023...",na,0
1602,7f66bb44,"[-22.455633, -25.794661, -26.954567, -22.83354...","[-26.070356, -22.093737, -21.577662, -24.53376...",na,0


In [7]:
plotmy3d(x_band_1[0, :, :], 'Ship')

우리는 배 형태를 시각화할 수 있는 해상도가 높지 않습니다. 그러나 CNN이 도움이 될 것입니다. 다음 링크에서 배와 빙하 분류에 대한 논문을 확인할 수 있습니다: [http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf](http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf) 그러나 해당 논문의 데이터는 훨씬 더 좋은 해상도를 가지고 있기 때문에 논문에서 사용한 CNN이 여기에는 적합하지 않다고 생각합니다.  

Keras를 이용하여 CNN을 구축해봅시다. 

### 3. Modeling  
https://zereight.tistory.com/227

In [61]:
# Keras 라이브러리
# from matplotlib import pyplot
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation
# from keras.layers import Input
# from keras.layers import GlobalMaxPooling2D
# from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
# from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [46]:
# 모델 정의
def getModel():
    # 모델 구축
    gmodel = Sequential()
    
    # Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    # Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    # Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    # Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    # Dense layer를 위해 데이터 펼치기
    gmodel.add(Flatten())
    
    # Dense Layer 1
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))
    
    # Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))
    
    # Dense Layer 3
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))
    
    myoptim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy', optimizer=myoptim, metrics=['accuracy'])
    gmodel.summary()
    return gmodel

💡 Conv2D: 필터를 이용하여 영상 특징 추출  
💡 MaxPooling2D: 영상에서의 사소한 변화가 특징 추출에 크게 영향을 미치지 않도록 함  
💡 Dropout: 과적합 방지를 위해 지정한 비율만큼 입력 뉴런 제외  

In [50]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

file_path = 'data/.model_weights.hdf5'
callbacks = get_callbacks(filepath=file_path, patience=5)

In [51]:
target_train = train['is_iceberg']
x_train_cv, x_valid, y_train_cv, y_valid = train_test_split(x_train, target_train, random_state=1, train_size=0.75)

In [53]:
import os
gmodel = getModel()
gmodel.fit(
    x_train_cv, y_train_cv,
    batch_size=24,
    epochs=50,
    verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks=callbacks
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 36, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 128)       1

### 4. Submission
점수: 0.30455 (LB)

In [54]:
gmodel.load_weights(filepath=file_path)
score = gmodel.evaluate(x_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 97ms/step - loss: 0.3520 - accuracy: 0.8379
Test loss: 0.3520285487174988
Test accuracy: 0.83790522813797


In [55]:
x_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
x_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])
x_test = np.concatenate([
    x_band_test_1[:, :, :, np.newaxis],
    x_band_test_2[:, :, :, np.newaxis],
    ((x_band_test_1 + x_band_test_2)/2)[:, :, :, np.newaxis]
], axis=-1)
predicted_test = gmodel.predict_proba(x_test)

C:\Users\PC\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning:

`model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.



In [57]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('data/submission_1_sub.csv', index=False)

### 5. Conclusion
점수를 높이기 위해 Speckle filtering, Indicence angle normalizaition 및 기타 전처리를 시도해 보았으나 잘 작동하지 않는 것 같습니다.  

이 커널로는 상위권에 들 수 없습니다. 테스트 데이터에는 8000개의 이미지가 있으므로 이것을 이용할 수 있습니다. 

## Keras Model for Beginners (0.210 on LB)+EDA+R&D

**✏ 필사 2회** 

### 1. Description
표류하는 빙산은 캐나다의 동부 해안과 같은 지역에서의 항해 및 활동에 위협을 줍니다.  

현재, 여러 기관과 회사들은 환경 조건 모니터링하고 빙산에 의한 위험을 평가하기 위해 항공 정찰과 해안 기반 지원 사업을 하고 있습니다. 그러나 날씨가 특히 험한 외딴 지역에서는 이런 방법들이 불가능하며, 실현 가능한 모니터링 방법은 위성을 통하는 것 뿐입니다.  

전세계에서 활동하고 있는 국제 에너지 회사 Statoil은 C-CORE 등의 기업들과 긴밀하게 협력해왔습니다. C-CORE는 30년이 넘는 위성 데이터를 가지고 있으며, 컴퓨터 비전 기반의 감시 시스템을 구축했습니다. 안전하고 효과적인 운영을 위해 Statoil은 가능한 한 빨리 위협적인 빙산을 더 정확하게 감지하고 구별하기 위한 머신러닝 사용법에 대한 새로운 관점을 얻고자 합니다.  

이 대회에서는 멀리서 감지된 타겟이 배인지 빙산인지를 자동으로 구별하는 알고리즘을 설계하는 것이 목표입니다. 개선은 안전 작업 환경 유지를 위한 비용을 절감하는데 도움이 될 것입니다.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
# from os.path import join as opj
import  matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# import pylab

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
train = pd.read_json('data/train.json')
test = pd.read_json('data/test.json')

### 2. Intro about the Data
Sentinet -1 위성은 지구로부터 약 680km 떨어져 있습니다. 특정 입사각에서 신호의 펄스를 보내고 다시 recoding합니다. 기본적으로 이러한 반사 신호를 backscatter(후방산란)라고 합니다. 우리에게 제공된 데이터는 다음 식에 의해 주어진 후방산란계수의 전형적인 형태입니다:  
$$\sigma_{o}(dB) = \beta_{o}(dB) + 10~log10[sin(ip)/sin(ic)]$$  

ip: 특정 픽셀의 입사각  
ic: 이미지 중심부의 입사각  
K: 상수  

우리는 데이터에서 $\sigma_{o}$를 바로 확인할 수 있습니다.

##### Now coming to the features of $\sigma_{o}$
일반적으로 $\sigma_{o}$는 신호가 산란되는 표면에 따라 달라집니다. 예를 들어, 특정 입사각에서는 다음과 같이 변합니다:  

ㅤ |WATER | SETTLEMENTS | AGRICULTURE | BARREN
-- | -- | -- | -- | --
**HH** | -27.001 | 2.70252 | -12.7952 | -17.25790909
**HV** | -28.035 | -20.2665 | -21.4471 | -20.019  

HH의 경우 값이 크게 변화하지만 HV는 그렇지 않습니다. **배에 대한 산란에 대한 데이터는 없지만 금속 물질이기 때문에 얼음과는 매우 달라야 합니다.**

##### WTF is HH HV?
Sentinel 위성은 RISTSAT(인도의 원격 감지 위성)와 같습니다. H polarization만 전송하고 V polarization은 전송하지 않습니다. H-ping이 산란되면 물질의 polarization이 바뀌고 H와 V가 혼합되어 돌아옵니다. **Sentinel 위성은 H-transmetter만 가지고 있기 때문에 돌아오는 신호는 HH와 HV 형태 뿐입니다.** Sentinel은 V-ping transmitter는 가지고 있지 않기 때문에 VV가 없는 이유에 대해서는 묻지 마십시오.  

먼저, 두 band를 모두 추출하고 평균값을 세 번째 채널로 사용하여 3-channel RGB equvalent를 생성하겠습니다.

In [3]:
train.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [5]:
x_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
x_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])
x_train = np.concatenate([
    x_band_1[:, :, :, np.newaxis],
    x_band_2[:, :, :, np.newaxis],
    ((x_band_1 + x_band_2)/2)[:, :, :, np.newaxis]
], axis=-1)

x_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
x_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])
x_test = np.concatenate([
    x_band_test_1[:, :, :, np.newaxis],
    x_band_test_2[:, :, :, np.newaxis],
    ((x_band_test_1 + x_band_test_2)/2)[:, :, :, np.newaxis]
], axis=-1)

In [23]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):
    data = [go.Surface(z=c)]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=600,
        height=500,
        margin=dict(l=65, r=50, b=65, t=90)
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

plotmy3d(x_band_1[12, :, :], 'iceberg')

레이더 데이터에서 빙산의 형태가 위와 같이 산처럼 될 것이라는 점을 기억해야 합니다. 실제 이미지가 아니라 레이더에 의한 산란이기 때문에 peak와 왜곡이 있을 수 있습니다. 배의 형태는 점과 같으며 가늘고 뾰족한 점이 될 것입니다. 여기서 구조적 차이가 발생하고, 우리는 CNN을 사용하여 차이점을 이용할 수 있습니다. 레이더의 backscatter를 이용하여 합성물을 만들 수 있다면 큰 도움이 될 것입니다.

In [24]:
train.loc[train['is_iceberg'] == 0, ]

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0
7,56929c16,"[-24.989119, -27.755224, -25.817074, -24.98927...","[-27.755173, -26.732174, -28.124943, -31.83772...",41.1769,0
...,...,...,...,...,...
1599,04e11240,"[-30.999878, -29.976866, -28.233906, -29.50732...","[-27.847719, -28.233864, -24.712077999999998, ...",na,0
1600,c7d6f6f8,"[-25.31155, -26.511555, -28.694487, -27.180115...","[-29.563713, -28.290375, -26.839405, -28.29046...",na,0
1601,bba1a0f1,"[-18.141895, -18.141844, -19.01737, -19.701599...","[-25.305355, -29.387701, -28.963863, -26.16023...",na,0
1602,7f66bb44,"[-22.455633, -25.794661, -26.954567, -22.83354...","[-26.070356, -22.093737, -21.577662, -24.53376...",na,0


In [25]:
plotmy3d(x_band_1[0, :, :], 'Ship')

우리는 배 형태를 시각화할 수 있는 해상도가 높지 않습니다. 그러나 CNN이 도움이 될 것입니다. 다음 링크에서 배와 빙하 분류에 대한 논문을 확인할 수 있습니다: [http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf](http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf) 그러나 해당 논문의 데이터는 훨씬 더 좋은 해상도를 가지고 있기 때문에 논문에서 사용한 CNN이 여기에는 적합하지 않다고 생각합니다.  

Keras를 이용하여 CNN을 구축해봅시다. 

### 3. Modeling  
https://zereight.tistory.com/227

In [26]:
# from matplotlib import pyplot
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation
# from keras.layers import Input
# from keras.layers import GlobalMaxPooling2D
# from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
# from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [62]:
def getModel():
    gmodel = Sequential()
    
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    gmodel.add(Flatten())
    
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))
    
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))
    
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))
    
    myoptim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(
        loss='binary_crossentropy',
        optimizer=myoptim,
        metrics=['accuracy']
    )
    gmodel.summary()
    return gmodel

In [63]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

file_path = 'data/.model_weights_2.hdf5'
callbacks = get_callbacks(filepath=file_path, patience=5)

In [64]:
target_train = train['is_iceberg']
x_train_cv, x_valid, y_train_cv, y_valid = train_test_split(x_train, target_train, random_state=1, train_size=0.75)

In [65]:
import os
gmodel = getModel()
gmodel.fit(
    x_train_cv, y_train_cv,
    batch_size=24,
    epochs=50,
    verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks=callbacks
)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 15, 15, 128)      

### 4. Submission
점수: 0.30455 (LB)

In [66]:
gmodel.load_weights(filepath=file_path)
score = gmodel.evaluate(x_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 99ms/step - loss: 0.3253 - accuracy: 0.8903
Test loss: 0.32531821727752686
Test accuracy: 0.8902742862701416


In [67]:
predicted_test = gmodel.predict_proba(x_test)

In [68]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('data/submission_1_sub2.csv', index=False)

### 5. Conclusion
점수를 높이기 위해 Speckle filtering, Indicence angle normalizaition 및 기타 전처리를 시도해 보았으나 잘 작동하지 않는 것 같습니다.  

이 커널로는 상위권에 들 수 없습니다. 테스트 데이터에는 8000개의 이미지가 있으므로 이것을 이용할 수 있습니다. 